In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv("./inputs/train.csv")
target = df["Price"]
df.drop("Price", inplace=True, axis=1)
df.drop("id", inplace=True, axis=1)
data = df
data

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg)
0,0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723
1,1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537
2,2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760
3,3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220
4,4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338
...,...,...,...,...,...,...,...,...,...,...
299995,299995,Adidas,Leather,Small,9.0,No,No,Tote,Blue,12.730812
299996,299996,Jansport,Leather,Large,6.0,No,Yes,Tote,Blue,26.633182
299997,299997,Puma,Canvas,Large,9.0,Yes,Yes,Backpack,Pink,11.898250
299998,299998,Adidas,Nylon,Small,1.0,No,Yes,Tote,Pink,6.175738


In [3]:
for i in data:
    if i != "Weight Capacity (kg)" and i != "Compartments":
        data[i] = LabelEncoder().fit_transform(data[i])

data

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg)
0,0,1,1,1,7.0,1,0,2,0,11.611723
1,1,1,0,2,10.0,1,1,1,3,27.078537
2,2,4,1,2,2.0,1,0,1,5,16.643760
3,3,2,2,2,8.0,1,0,1,3,12.937220
4,4,0,0,1,1.0,1,1,1,3,17.749338
...,...,...,...,...,...,...,...,...,...,...
299995,299995,0,1,2,9.0,0,0,2,1,12.730812
299996,299996,1,1,0,6.0,0,1,2,1,26.633182
299997,299997,3,0,0,9.0,1,1,0,4,11.898250
299998,299998,0,2,2,1.0,0,1,2,4,6.175738


In [4]:
for i in data:
    if sorted(data[i].unique())[-1] < 8:
        data[i] = np.where(data[i] == sorted(data[i].unique())[-1], np.nan, data[i])

%store data
data

Stored 'data' (DataFrame)


,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg)
0,0,1.0,1.0,1.0,7.0,1.0,0.0,2.0,0.0,11.611723
1,1,1.0,0.0,2.0,10.0,1.0,1.0,1.0,3.0,27.078537
2,2,4.0,1.0,2.0,2.0,1.0,0.0,1.0,5.0,16.643760
3,3,2.0,2.0,2.0,8.0,1.0,0.0,1.0,3.0,12.937220
4,4,0.0,0.0,1.0,1.0,1.0,1.0,1.0,3.0,17.749338
...,...,...,...,...,...,...,...,...,...,...
299995,299995,0.0,1.0,2.0,9.0,0.0,0.0,2.0,1.0,12.730812
299996,299996,1.0,1.0,0.0,6.0,0.0,1.0,2.0,1.0,26.633182
299997,299997,3.0,0.0,0.0,9.0,1.0,1.0,0.0,4.0,11.898250
299998,299998,0.0,2.0,2.0,1.0,0.0,1.0,2.0,4.0,6.175738


In [5]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [13]:
model = CatBoostRegressor(n_estimators=1000, learning_rate=0.01, max_depth=6, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mean = root_mean_squared_error(y_test, y_pred)
print(mean)

0:	learn: 39.0625457	total: 18.4ms	remaining: 18.4s
1:	learn: 39.0620256	total: 39.9ms	remaining: 19.9s
2:	learn: 39.0614744	total: 58.4ms	remaining: 19.4s
3:	learn: 39.0609175	total: 74.5ms	remaining: 18.6s
4:	learn: 39.0604692	total: 91.6ms	remaining: 18.2s
5:	learn: 39.0599919	total: 110ms	remaining: 18.3s
6:	learn: 39.0595513	total: 129ms	remaining: 18.3s
7:	learn: 39.0590796	total: 146ms	remaining: 18.1s
8:	learn: 39.0586861	total: 164ms	remaining: 18s
9:	learn: 39.0582827	total: 182ms	remaining: 18.1s
10:	learn: 39.0578748	total: 202ms	remaining: 18.2s
11:	learn: 39.0574205	total: 221ms	remaining: 18.2s
12:	learn: 39.0569691	total: 246ms	remaining: 18.7s
13:	learn: 39.0566124	total: 271ms	remaining: 19.1s
14:	learn: 39.0561003	total: 293ms	remaining: 19.3s
15:	learn: 39.0557173	total: 314ms	remaining: 19.3s
16:	learn: 39.0552562	total: 335ms	remaining: 19.4s
17:	learn: 39.0548981	total: 354ms	remaining: 19.3s
18:	learn: 39.0544652	total: 373ms	remaining: 19.2s
19:	learn: 39.05408